In [16]:
%load_ext dotenv
%dotenv 

# What are we doing?

## Objectives 


* Build a data pipeline that downloads price data from the internet, stores it locally, transforms it into return data, and stores the feature set.
    - Getting the data.
    - Schemas and index in dask.

* Explore the parquet format.
    - Reading and writing parquet files.
    - Read datasets that are stored in distributed files.
    - Discuss dask vs pandas as a small example of big vs small data.
    
* Discuss the use of environment variables for settings.
* Discuss how to use Jupyter notebooks and source code concurrently. 
* Logging and using a standard logger.

## About the Data

+ We will download the prices for a list of stocks.
+ The source is Yahoo Finance and we will use the API provided by the library yfinance.


## Medallion Architecture

+ The architecture that we are thinking about is called Medallion by [DataBricks](https://www.databricks.com/glossary/medallion-architecture). It is an ELT type of thinking, although our data is well-structured.

![Medallion Architecture (DataBicks)](./images/02_medallion_architecture.png)

+ In our case, we would like to optimize the number of times that we download data from the internet. 
+ Ultimately, we will build a pipeline manager class that will help us control the process of obtaining and transforming our data.

![](./images/02_target_pipeline_manager.png)

# Download Data from Yahoo Finance

Yahoo Finance provides information about public stocks in different markets. The library yfinance gives us access to a fair bit of the data in Yahoo Finance. 

These steps are based on the instructions in:

+ [yfinance documentation](https://pypi.org/project/yfinance/)
+ [Tutorial in geeksforgeeks.org](https://www.geeksforgeeks.org/get-financial-data-from-yahoo-finance-with-python/)


+ If required, install: `python -m pip install yfinance`.
+ To download the price history of a stock, first use the following setup:


In [17]:
import pandas as pd
import yfinance as yf
import os
import sys

sys.path.append(os.getenv('SRC_DIR'))

A few things to notice in the code chunk above:

+ Libraries are ordered from high-level to low-level libraries from the package manager (pip in this case, but could be conda, poetry, etc.)
+ The command `sys.path.append("../05_src/)` will add the `../05_src/` directory to the path in the Notebook's kernel. This way, we can use our modules as part of the notebook.
+ Local modules are imported at the end. 
+ The function `get_logger()` is called with `__name__` as recommended by the documentation.

Now, to download the historical price data for a stock, we could use:

In [18]:
px = yf.download('AAPL', start = "2013-12-01", end = "2024-02-01")
px

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2013-12-02 00:00:00+00:00,17.175098,19.686787,20.154642,19.672144,19.928572,472544800
2013-12-03 00:00:00+00:00,17.645269,20.225714,20.227858,19.917143,19.939285,450968000
2013-12-04 00:00:00+00:00,17.604145,20.178572,20.328215,20.029285,20.196428,377809600
2013-12-05 00:00:00+00:00,17.694498,20.282143,20.540714,20.228930,20.451786,447580000
2013-12-06 00:00:00+00:00,17.448971,20.000713,20.241072,19.984644,20.206785,344352400
...,...,...,...,...,...,...
2024-01-25 00:00:00+00:00,193.223389,194.169998,196.270004,193.110001,195.220001,54822100
2024-01-26 00:00:00+00:00,191.481918,192.419998,194.759995,191.940002,194.270004,44594000


## Parametrize the download

+ Generally, we will look to separate every parameter and setting from functions.
+ If we had a few stocks, we could cycle through them. We need a place to store the list of tickers (a db or file, for example).
+ Store a csv file with a few stock tickers. The location of the file is a setting, the contents of this file are parameters.
+ Use **environment variables** to pass parameters.

Start by getting a sample of Information Technology stock tickers by applying subindexing and converting the "ticker" column from a pandas object to a list.

In [19]:
# Load all tickers
ticker_file = "/Users/atulkalra/Desktop/DSI Assignment/production/05_src/data/tickers/sample_tickers.csv"
os.environ["TICKERS"] = ticker_file  # Set the environment variable
tickers = pd.read_csv(os.getenv("TICKERS"))

We can subset our ticker data set using standard indexing techniques. A good reference for this type of data manipulation is Panda's [Documentation](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-and-selecting-data) and [Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook-selection).

In [20]:
idx_tech = tickers['GICS Sector'] == 'Information Technology'
tech_sector = tickers[idx_tech]

From the subset data frame, select one column and convert to list.

In [21]:
tech_tickers = tech_sector['ticker'].to_list()

In [22]:
tech_raw_dt = yf.download(tech_tickers, start = "2000-01-01", end = "2025-01-26")


[*********************100%***********************]  4 of 4 completed


The data that we downloaded combines several stocks and prices into a single row. We want to parse this arrangement into a dataframe that contains observations about a single stock on a given day per row. To do this, we can use the function [`stack()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html) and re-arrange the indices.

In [23]:
# First, check what tech_raw_dt.stack() looks like.
tech_raw_dt.stack(future_stack=True).reset_index()

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2000-01-03 00:00:00+00:00,AAPL,0.843076,0.999442,1.004464,0.907924,0.936384,535796800.0
1,2000-01-03 00:00:00+00:00,CDW,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-03 00:00:00+00:00,MSFT,35.864075,58.281250,59.312500,56.000000,58.687500,53228400.0
3,2000-01-03 00:00:00+00:00,NVDA,0.089440,0.097526,0.099219,0.091927,0.098438,300912000.0
4,2000-01-04 00:00:00+00:00,AAPL,0.771997,0.915179,0.987723,0.903460,0.966518,512377600.0
...,...,...,...,...,...,...,...,...
25211,2025-01-23 00:00:00+00:00,NVDA,147.220001,147.220001,147.229996,143.720001,145.050003,155915500.0
25212,2025-01-24 00:00:00+00:00,AAPL,222.779999,222.779999,225.630005,221.410004,224.779999,54697900.0
25213,2025-01-24 00:00:00+00:00,CDW,195.130005,195.130005,198.470001,194.889999,198.059998,940600.0
25214,2025-01-24 00:00:00+00:00,MSFT,444.059998,444.059998,446.649994,441.399994,445.160004,15549500.0


In [24]:
tech_dt = (tech_raw_dt
           .stack(future_stack=True)
           .reset_index()
           .sort_values(['Ticker', 'Date']))
tech_dt.columns.name = None


# Storing Data in CSV



+ We have some data. How do we store it?
+ We can compare two options, CSV and Parqruet, by measuring their performance:

    - Time to save.
    - Space required.

In [25]:
def get_dir_size(path='.'):
    '''Returns the total size of files contained in path.'''
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

In [26]:
import time

In [27]:
# Set the TEMP_DATA environment variable if it's not already set
temp = os.getenv("TEMP_DATA", "/Users/atulkalra/Desktop/temp_directory")  # Use a writable directory
os.makedirs(temp, exist_ok=True)
stock_path = os.path.join(temp, "stock_px.csv")

In [28]:
start = time.time()
tech_dt.to_csv(stock_path, index = False)
end = time.time()

print(f'Writing to dt ({tech_dt.shape})csv took {end - start} seconds.')
print(f'Csv file size { os.path.getsize(stock_path)*1e-6 } MB')

Writing to dt ((25216, 8))csv took 0.4960620403289795 seconds.
Csv file size 3.0232859999999997 MB


## Save Data to Parquet

### Dask 

We can work with with large data sets and parquet files. In fact, recent versions of pandas support pyarrow data types and future versions will require a pyarrow backend. The pyarrow library is an interface between Python and the Appache Arrow project. The [parquet data format](https://parquet.apache.org/) and [Arrow](https://arrow.apache.org/docs/python/parquet.html) are projects of the Apache Foundation.

However, Dask is much more than an interface to Arrow: Dask provides parallel and distributed computing on pandas-like dataframes. It is also relatively easy to use, bridging a gap between pandas and Spark. 

In [29]:
import os
os.environ['DASK_ROOT'] = '/your/dask/cache/directory'

In [30]:
import os
os.environ['DASK_ROOT'] = '/your/dask/cache/directory'
os.environ['DASK_TEMP'] = '/your/dask/temp/directory'

In [31]:
import os
os.environ["DASK_CACHE_DIR"] = "/tmp/dask_cache"
os.environ["DASK_WORKER_TEMP_DIR"] = "/tmp/dask_worker_temp"
os.environ["DASK_DISTRIBUTED_SCHEDULER"] = "tcp://localhost:8786"

In [32]:
import dask
print(dask.__version__)

2024.8.0


In [33]:
import os
print("DASK_TEMP_DIR:", os.getenv("DASK_TEMP_DIR"))
print("DASK_WORKER_TEMP_DIR:", os.getenv("DASK_WORKER_TEMP_DIR"))
print("DASK_CACHE_DIR:", os.getenv("DASK_CACHE_DIR"))

DASK_TEMP_DIR: /tmp/dask_temp
DASK_WORKER_TEMP_DIR: /tmp/dask_worker_temp
DASK_CACHE_DIR: /tmp/dask_cache


In [34]:
import os

os.environ["DASK_TEMP_DIR"] = "/tmp/dask_temp"
os.environ["DASK_WORKER_TEMP_DIR"] = "/tmp/dask_worker_temp"
os.environ["DASK_CACHE_DIR"] = "/tmp/dask_cache"

import dask
print("DASK_TEMP_DIR:", os.getenv("DASK_TEMP_DIR"))

DASK_TEMP_DIR: /tmp/dask_temp


In [35]:
import dask
print("DASK_TEMP_DIR:", os.getenv("DASK_TEMP_DIR"))

DASK_TEMP_DIR: /tmp/dask_temp


In [36]:
import dask.dataframe as dd

In [37]:
import dask.dataframe as dd
import os
import time

# Assuming tech_dt and tech_tickers are defined previously
px_dd = dd.from_pandas(tech_dt, npartitions = len(tech_tickers))
parquet_path = os.path.join(temp, "stock_px.parquet")

start = time.time()
px_dd.to_parquet(parquet_path, engine = "pyarrow")
end = time.time()

print(f'Writing dd ({tech_dt.shape}) to parquet took {end - start} seconds.')
print(f'Parquet file size {get_dir_size(parquet_path)*1e-6} MB')

Writing dd ((25216, 8)) to parquet took 0.9748728275299072 seconds.
Parquet file size 1.115143 MB


In [38]:
px_dd = dd.from_pandas(tech_dt, npartitions = len(tech_tickers))
parquet_path = os.path.join(temp, "stock_px.parquet")

start = time.time()
px_dd.to_parquet(parquet_path, engine = "pyarrow")
end = time.time()

print(f'Writing dd ({tech_dt.shape}) to parquet took {end - start} seconds.')
print(f'Parquet file size { get_dir_size(parquet_path)*1e-6 } MB')

Writing dd ((25216, 8)) to parquet took 0.09913992881774902 seconds.
Parquet file size 1.115143 MB


### Parquet files and Dask Dataframes

+ Parquet files are immutable: once written, they cannot be modified.
+ Dask DataFrames are a useful implementation to manipulate data stored in parquets.
+ Parquet and Dask are not the same: parquet is a file format that can be accessed by many applications and programming languages (Python, R, PowerBI, etc.), while Dask is a package in Python to work with large datasets using distributed computation.
+ **Dask is not for everything** (see [Dask DataFrames Best Practices](https://docs.dask.org/en/stable/dataframe-best-practices.html)). 

    - Consider cases suchas small to large joins, where the small dataframe fits in memory, but the large one does not. 
    - If possible, use pandas: reduce, then use pandas.
    - Pandas performance tips apply to Dask.
    - Use the index: it is beneficial to have a well-defined index in Dask DataFrames, as it may speed up searching (filtering) the data. A one-dimensional index is allowed.
    - Avoid (or minimize) full-data shuffling: indexing is an expensive operations. 
    - Some joins are more expensive than others. 

        * Not expensive:

            - Join a Dask DataFrame with a pandas DataFrame.
            - Join a Dask DataFrame with another Dask DataFrame of a single partition.
            - Join Dask DataFrames along their indexes.

        * Expensive:

            - Join Dask DataFrames along columns that are not their index.


# How do we store prices?

+ We can store our data as a single blob. This can be difficult to maintain, especially because parquet files are immutable.
+ Strategy: organize data files by ticker and date. Update only latest month.



In [39]:
# CLean up before start
PRICE_DATA = os.getenv("PRICE_DATA")
import shutil
if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)

In [40]:
for ticker in tech_dt['Ticker'].unique():
    ticker_dt = tech_dt[tech_dt['Ticker'] == ticker]
    ticker_dt = ticker_dt.assign(Year = ticker_dt.Date.dt.year)
    for yr in ticker_dt['Year'].unique():
        yr_dd = dd.from_pandas(ticker_dt[ticker_dt['Year'] == yr],2)
        yr_path = os.path.join(PRICE_DATA, ticker, f"{ticker}_{yr}")
        os.makedirs(os.path.dirname(yr_path), exist_ok=True)
        yr_dd.to_parquet(yr_path, engine = "pyarrow")
    

Why would we want to store data this way?

+ Easier to maintain. We do not update old data, only recent data.
+ We can also access all files as follows.

# Load, Transform and Save 

## Load

+ Parquet files can be read individually or as a collection.
+ `dd.read_parquet()` can take a list (collection) of files as input.
+ Use `glob` to get the collection of files.

In [41]:
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

## Transform

+ This transformation step will create a *Features* data set. In our case, features will be stock returns (we obtained prices).
+ Dask dataframes work like pandas dataframes: in particular, we can perform groupby and apply operations.
+ Notice the use of [an anonymous (lambda) function](https://realpython.com/python-lambda/) in the apply statement.

In [42]:
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

/var/folders/w1/md41q_qx3p3bscs6dwnb2_bh0000gn/T/ipykernel_5314/3086392435.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


In [43]:
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

## Lazy Exection

What does `dd_rets` contain?

In [44]:
dd_rets

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns
npartitions=4,,,,,,,,,,
AAPL,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64
CDW,...,...,...,...,...,...,...,...,...,...
MSFT,...,...,...,...,...,...,...,...,...,...
NVDA,...,...,...,...,...,...,...,...,...,...
NVDA,...,...,...,...,...,...,...,...,...,...


+ Dask is a lazy execution framework: commands will not execute until they are required. 
+ To trigger an execution in dask use `.compute()`.

In [45]:
dd_rets.compute()

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns
Ticker,,,,,,,,,,
AAPL,2000-01-03 00:00:00+00:00,0.843076,0.999442,1.004464,0.907924,0.936384,535796800.0,2000,NaN,NaN
AAPL,2000-01-04 00:00:00+00:00,0.771997,0.915179,0.987723,0.903460,0.966518,512377600.0,2000,0.999442,-0.084310
AAPL,2000-01-05 00:00:00+00:00,0.783294,0.928571,0.987165,0.919643,0.926339,778321600.0,2000,0.915179,0.014633
AAPL,2000-01-06 00:00:00+00:00,0.715509,0.848214,0.955357,0.848214,0.947545,767972800.0,2000,0.928571,-0.086538
AAPL,2000-01-07 00:00:00+00:00,0.749401,0.888393,0.901786,0.852679,0.861607,460734400.0,2000,0.848214,0.047369
...,...,...,...,...,...,...,...,...,...,...
NVDA,2025-01-17 00:00:00+00:00,137.710007,137.710007,138.500000,135.460007,136.690002,201188800.0,2025,133.570007,0.030995
NVDA,2025-01-21 00:00:00+00:00,140.830002,140.830002,141.830002,137.089996,139.160004,197749000.0,2025,137.710007,0.022656
NVDA,2025-01-22 00:00:00+00:00,147.070007,147.070007,147.789993,143.669998,144.660004,237651400.0,2025,140.830002,0.044309


## Save

+ Apply transformations to calculate daily returns
+ Store the enriched data, the silver dataset, in a new directory.
+ Should we keep the same namespace? All columns?

In [46]:
# CLean up before save
FEATURES_DATA = os.getenv("FEATURES_DATA")
if os.path.exists(FEATURES_DATA):
    shutil.rmtree(FEATURES_DATA)
dd_rets.to_parquet(FEATURES_DATA, overwrite = True)

# Optional: from Jupyter to Command Line

+ We have drafted our code in a Jupyter Notebook. 
+ Finalized code should be written in Python modules.

## Object Oriented vs Functional Programming

+ We can use classes to keep parameters and functions together.
+ We *could* use Object Oriented Programming, but parallelization of data manipulation and modelling tasks benefit from *Functional Programming*.
+ An Idea: 

    - [Data Oriented Programming](https://blog.klipse.tech/dop/2022/06/22/principles-of-dop.html).
    - Use the class to bundle together parameters and functions.
    - Use stateless operations and treat all data objects as immutable (we do not modify them, we overwrite them).
    - Take advantage of [`@staticmethod`](https://realpython.com/instance-class-and-static-methods-demystified/).

The code is in `./05_src/`.

Our original design was:

![](./images/02_target_pipeline_manager.png)



The `DataManager` class in `./05_src/data_manager.py` is a simple implementation of the ideas and code discussed in this notebook. The lines below will download data for about 500 stocks from the S&P500. Using this data a few features will be created and stored in the features data set.

First, instantiate an object of class `DataManager`.

In [47]:
from data_manager import DataManager
dm = DataManager()

Download all prices.

In [48]:
dm.download_all()

2025-02-10 18:02:12,198, data_manager.py, 42, INFO, Getting price data for all tickers.
2025-02-10 18:02:12,202, data_manager.py, 51, INFO, Getting tickers from /Users/atulkalra/Desktop/DSI Assignment/production/05_src/data/tickers/sample_tickers.csv
2025-02-10 18:02:12,226, data_manager.py, 57, INFO, Processing all tickers
2025-02-10 18:02:12,228, data_manager.py, 70, INFO, Processing ticker ['AAPL', 'GOOGL', 'GOOG', 'NVDA', 'MSFT', 'AMZN', 'FB', 'TSLA', 'ABNB', 'WBA', 'WMT', 'DIS', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'CAT', 'CBOE', 'CBRE', 'CDW']
2025-02-10 18:02:12,229, data_manager.py, 100, INFO, Getting stock price data for ['AAPL', 'GOOGL', 'GOOG', 'NVDA', 'MSFT', 'AMZN', 'FB', 'TSLA', 'ABNB', 'WBA', 'WMT', 'DIS', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'CAT', 'CBOE', 'CBRE', 'CDW'] from 2000-01-01 to 2025-02-10
[*********************100%***********************]  21 of 21 completed

1 Failed download:
['FB']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
2025-02-10 

Finally, add features to the data set and save to a *feature store*.

In [49]:
dm.featurize()

2025-02-10 18:02:38,627, data_manager.py, 114, INFO, Creating features data.
2025-02-10 18:02:38,633, data_manager.py, 124, INFO, Loading price data from ../../05_src/data/prices/
2025-02-10 18:02:38,790, data_manager.py, 133, INFO, Creating features
/Users/atulkalra/Desktop/DSI Assignment/production/01_materials/labs/../../05_src/data_manager.py:136: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  features = (price_dd.groupby('Ticker', group_keys=False)
2025-02-10 18:02:38,799, data_manager.py, 158, INFO, Saving features to ../../05_src/data/features/stock_features.parquet
